In [3]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv


Autosaving every 180 seconds


In [4]:
######################################################
#### POST CNN CLASSIFIER COUNT CSUAS, LENGTHS, ETC ###
######################################################

# default labels
names = ['female','male','pup1','pup2']
clrs = ['red','blue','cyan','green','yellow']

# select CNN output file
fname_classification = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz'
data = np.load(fname_classification, allow_pickle=True)

# load the vals...
vals = data['vals']
print ("# of labeled frames: ", vals.shape)

# load the predicted classes
classes = data['classes']

print (" # of labeled segments: ", np.unique(vals).shape)
n_animals = 5
single_ids = 0
threshold = 0.7
correct_ids = 0


plotting = False
if plotting:
    fig=plt.figure()
percent_array = []
lengths = []
for k in range(5):
    lengths.append([])
# 
ctr=0
vals_in = np.unique(vals)
for dir_ in vals_in:
    idx = np.where(vals==dir_)[0]
    
    #print (idx.shape)
    temp_classes = classes[idx]
    #     
        
    if plotting:
        if ctr>29:
            break

        if idx.shape[0]<30:
            continue

        ax=plt.subplot(5,6,ctr+1)
    #y = np.histogram(temp_classes, np.arange(-0.5,3.5))
    
    temp_ctr=0
    id_ = None
    for k in np.unique(temp_classes):
        idx=np.where(temp_classes==k)[0]
        #print (names[k], idx.shape)
        if plotting:
            plt.bar(k,idx.shape[0],.9, color=clrs[k])
    
        if idx.shape[0]>temp_ctr:
            temp_ctr = idx.shape[0]
            id_ = k
            
    lengths[id_].append(temp_classes.shape[0])

    # compute % for best match
    percent = round((np.where(temp_classes==id_)[0].shape[0])/temp_classes.shape[0],2)
    percent_array.append(percent)
    
    #
    if plotting:
        plt.title("seg: "+str(int(dir_))+", "+names[id_]+ " "+str(round(percent*100,2))+"%", fontsize=8)
        plt.xticks([])
        plt.xlim(-0.5,4.5)

            
    if percent>threshold:
        single_ids+=1
        correct_ids+=np.where(temp_classes==id_)[0].shape[0]
        
    ctr+=1
    #if ctr==1:
    #    break
    
print ("% correct (segments): ", single_ids/ np.unique(vals).shape[0])
print ("% correct (individual frame): ", correct_ids/ vals.shape[0])
if plotting:
    plt.show()
    

# of lableed frames:  (292353,)
 # of labeled segments:  (12467,)
% correct (segments):  0.8520093045640491
% correct (individual frame):  0.6277992700605091


In [13]:
####################################################
########### DISTRIBUTION: LENGTHS OF CSUA  #########
####################################################
fig = plt.figure()
width=9
width2 =  0.19
frame_rate=25
rec_len = 3600
#plt.title("Distribution of lengths of segment (total rec: "+str(rec_len)+" sec)")
for k in range(4):
    temp = np.array(lengths[k])/25
    y = np.histogram(temp, np.arange(0,rec_len,1))

    idx = np.where(classes==k)[0]
    #plt.scatter(times[idx]/25., classes[idx], c=clrs[k])
    total_dur = np.unique(times[idx]).shape[0]/frame_rate
    
    
    if False:
        if k<1:
            plt.bar(y[1][:-1], y[0], width, color=clrs[k])
            dataset_old = y[0].copy()
        else:
            plt.bar(y[1][:-1], y[0], width, bottom=dataset_old, color=clrs[k])

            dataset_old = y[0].copy()+dataset_old
    else:
        #plt.bar(y[1][:-1]+width2*k+0.5, y[0], width2, color=clrs[k])
        plt.plot(y[1][:-1]+width2*k+0.5, y[0], color=clrs[k], 
                 label=names[k]+" "+str(total_dur)+"sec")

        
        
    #p2 = plt.bar(ind, dataset2, width, bottom=dataset1, color='b')
plt.legend(fontsize=20)
plt.semilogy(bottom=1E-2)
plt.semilogx(bottom=1E-2)
#plt.xlim(0,50)
plt.xlabel("Length of time segment (seconds)",fontsize=20)
plt.ylabel("# of segments",fontsize=20)

plt.show()

/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.


In [7]:
####################################################
########## HISTOGRAM EXAMPLES OF PURE CSUAS ########
####################################################

print (np.unique(vals).shape)

segs = np.random.choice(np.arange(10000), 5000)
segs = np.arange(np.unique(vals).shape[0])
ctr=0
total_frames = 0
all_frames = 0
min_length_CSUA = 50
purity_threshold = 0.9
for seg in segs:
    # find the classifier values for that segment
    idx = np.where(vals==seg)[0]
    all_frames += idx.shape[0]
    if idx.shape[0]<min_length_CSUA:
        continue
        
    total_frames+=idx.shape[0]
    #print (classes[idx], classes[idx].shape)
    y=np.histogram(classes[idx], np.arange(-0.5, 5.5))

    if (np.max(y[0])/classes[idx].shape[0])<purity_threshold:
        continue        
   
    #plt.title(str(seg)+" "+str(np.max(y[0])/classes[idx].shape[0]))
    if ctr<100:
        ax=plt.subplot(10,10,ctr+1)
        plt.bar(y[1][:-1]+0.5, y[0],.9)
        plt.xticks([])
    ctr+=1

        
print ("# of segments: ", ctr, "  # of frames; ", total_frames)
plt.suptitle("Example of 100 CSUAs > "+str(min_length_CSUA)+ " frames (@25FPS)  and > "+str(purity_threshold)+ " pure"+
          "\n All data # of segments: "+ str(ctr)+ " (total: "+str(segs.shape[0])+ ")"+
             "\n# of additional images (potentially to train on) "+ str(total_frames) + 
             "(of total images; "+str(all_frames)+")", fontsize=20)
plt.show()

(12467,)
# of segments:  292   # of frames;  232940


In [25]:
# FIND CLASSIFIER VALUES FOR EACH CC FRAME
import glob


def get_animal_ids(frame,
                  chain_ids,
                  vals,
                  classes,
                  comments=False):
    #frame = 988
    #print ("FRAME: ", frame)
    #print ("classes: ", classes.shape)
    
    dirs = chain_ids[frame]

    if comments:
        print ("DIRS: ", dirs)
    
    animal_ids = []
    animal_ids2 = []
    threshold_purity1 = 0.5
    threshold_purity2 = 0.7

    # HUNGARIAN ALGORITHM BASED SEARCH FOR MOST CORRECT IDS
    best_ids_cts = np.zeros((5,5),'float32')
    animals3 = []
    for ctr_dir, dir_ in enumerate(dirs):
        #print ("2nd Loop: ", dir_)
        if np.isnan(dir_)==False:
            # find the classifier values for that segment
            idx = np.where(vals==dir_)[0]

            # find predicted id
            fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npy',recursive = False))

            ctr_chain = 0
            for fname in fnames:
                if ('frame_'+str(frame).zfill(7)) in fname:
                     break 
                ctr_chain+=1
        
            animal_id3 = classes[idx][ctr_chain]
            animals3.append(animal_id3)
            
            idx5 = np.where(classes[idx]<1E10)[0]
            classes_local = classes[idx[idx5]]
            
            classes_unique = np.unique(classes_local, return_counts=True)
            #print ("best id: ", best_id)
            
            best_ids_cts[ctr_dir, classes_unique[0]]=classes_unique[1]/idx.shape[0]
        else:
            animals3.append(None)
    
    # FIRST LOOP TO CHECK ANY GREAT MATCHES AND ZERO OUT EVERYTHING ELSE
    for k1 in range(5):
        temp = best_ids_cts[k1]
        max_val = np.max(temp)
        if max_val>threshold_purity2:
            idx8 = np.argmax(temp)
            best_ids_cts[:,idx8]=0
            best_ids_cts[k1,idx8]=max_val
    if comments:
        print (best_ids_cts)
    # RENORMALIZE PROBABILITIES AFTER ZEROING OUT
    for k1 in range(5):
        sum_ = best_ids_cts[k1].sum()
        if sum_>0:
            best_ids_cts[k1]*= 1./sum_
    if comments:
        print (best_ids_cts)    
        
    # FINAL LOOP TO GRAB WHATEVER IS LEFT
    for k1 in range(5):
        max_val = np.max(best_ids_cts[k1])
        if max_val==0: # or max_val<threshold_purity1:
            animal_ids.append(None)
            animal_ids2.append(animals3[k1])
            continue
        else:
            idx9 = np.argmax(best_ids_cts[k1])
            #print ("idx9: ", idx9, classes_unique[0])
            animal_ids.append(idx9)
            animal_ids2.append(animals3[k1])
            
            best_ids_cts[:,idx9]= 0
            # must zero out all the others
            
    if comments:
        print (best_ids_cts)           
    return animal_ids, animal_ids2, dirs

In [26]:
root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'


#
fname_chain_ids = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988_chain_id.npy'
chain_ids = np.load(fname_chain_ids)
print (chain_ids.shape)


(89988, 5)


In [28]:
#################################################################
###### MAKE HUNGARIAN ALGORITHM ASSEMBLED ANIMAL VIDEOS #########
#################################################################

data = np.load(fname_classification, allow_pickle=True)
vals = data['vals']
frames = np.hstack(data['frames_array'])
print ("# of lableed frames: ", vals.shape)
classes = data['classes']
#names = ['female','male','pup1','pup2','block']
#clrs = ['red','blue','cyan','green','yellow']


# Add fixed labels to video:
fname_traces_inferences = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
reassembled = np.load(fname_traces_inferences)
#reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
tracesx_re = reassembled['tracesx'].T
tracesy_re = reassembled['tracesy'].T
print ("Lenght of tracessx: ", tracesx_re.shape)

# OPTIONAL MAKE VIDEO TO REVIEW ASSEMBLED VS. INFERENCE LABELS (PRE-FIX)
# colors have weird inversion; red is blue and cyan is yellow
#colors_4 = ['blue','red','cyan','green','pink','orange']

#          pup1     pup2    female  male
colors_4= ['orange','green', 'blue', 'red', 'cyan']
colors_5= ['red','blue', 'cyan', 'green', 'yellow']

video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
#video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale
dot_size2 = 4//scale

#

#dot_size = 10//scale
# setup cutoff 
pcutoff = 0.01

# go through first videos
from tqdm import trange

start = 1
end = tracesx_re.shape[0]
#end = 1001
n_networks = 5
comments=False

out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_CNN_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

# LOOP 
font = cv2.FONT_HERSHEY_PLAIN
for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
        
    animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                chain_ids,
                                vals,
                                classes,
                                comments)
    shift=20
    #titles = np.zeros(n_networks)
    for k in range(0, 14*n_networks,14):
    #for k in range(14*n_networks):
        y = tracesx_re[n,k:k+14]
        x = tracesy_re[n,k:k+14]
        
        #if animal_ids[k//14]==None:
        #    continue
            
        idx = np.where(np.isnan(x))[0]
        x = np.delete(x,idx,0)
        y = np.delete(y,idx,0)
        idx = np.where(x==0)[0]
        x = np.delete(x,idx,0)
        y = np.delete(y,idx,0)
        
        #if np.isnan(x) or np.isnan(y):
        #    continue
        
        if x.shape[0]==0:
            continue
        #print (x)
        x1=np.int32(x)//scale
        y1=np.int32(y)//scale
        
        flag=True
        for x, y in zip(x1,y1):
            if animal_ids[k//14]==1E10:
                continue
                
            if animal_ids[k//14]!= None:
                frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                    #matplotlib.colors.to_rgb(colors_4[k//14]))*255.).astype('uint8')
                    #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')
                    matplotlib.colors.to_rgb(colors_5[animal_ids[k//14]]))*255.).astype('uint8')

            if animal_ids2[k//14]!= None:
                frame[x-dot_size2+shift:x+shift+dot_size2,y-dot_size2:y+dot_size2]= (np.float32(
                    #matplotlib.colors.to_rgb(colors_4[k//14]))*255.).astype('uint8')
                    #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')
                    matplotlib.colors.to_rgb(colors_5[animal_ids2[k//14]]))*255.).astype('uint8')
        
        #print (colors_4[k])
        #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
        #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')

        #if titles[k//14]==0:
            if flag:
                cv2.putText(frame, str(dirs[k//14]), (y, x), font, 2, (255, 255, 0), 5)
                flag=False
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()


  0%|          | 0/89987 [00:00<?, ?it/s]

# of lableed frames:  (292353,)
Lenght of tracessx:  (89988, 70)


100%|██████████| 89987/89987 [49:17<00:00, 30.43it/s]  


In [47]:
#################################################################
###### GET HUNGARIAN ALGORITHM ASSEMBLED DATA WITHOUT VIDEO #####
#################################################################

data = np.load(fname_classification, allow_pickle=True)
vals = data['vals']
frames = np.hstack(data['frames_array'])
print ("# of lableed frames: ", vals.shape)
classes = data['classes']
#names = ['female','male','pup1','pup2','block']
#clrs = ['red','blue','cyan','green','yellow']


# Add fixed labels to video:
fname_traces_inferences = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
reassembled = np.load(fname_traces_inferences)
#reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
tracesx_re = reassembled['tracesx'].T
tracesy_re = reassembled['tracesy'].T
print ("Lenght of tracessx: ", tracesx_re.shape)

# go through first videos
from tqdm import trange

start = 1
end = tracesx_re.shape[0]
#end = 1000
comments=False
n_networks = 5

animal_arrays = np.zeros((n_networks, tracesx_re.shape[0], 2), 'float32')
for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    
    animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                chain_ids,
                                vals,
                                classes,
                                comments)
    
    #print ("animal_ids: ", animal_ids)
    for k in range(0, 14*n_networks,14):
        y = tracesx_re[n,k:k+14]
        x = tracesy_re[n,k:k+14]
        
        idx = np.where(x!=0)[0]
        x = x[idx]
        y = y[idx]
        if x.shape[0]==0:
            x_ave = None
            y_ave = None
        else:
            x_ave = np.nanmean(x)
            y_ave = np.nanmean(y)
        
        if animal_ids[k//14] is None:
            continue
        animal_arrays[animal_ids[k//14], n] = x_ave, y_ave

  0%|          | 6/89987 [00:00<27:03, 55.42it/s]

# of lableed frames:  (292353,)
Lenght of tracessx:  (89988, 70)


100%|██████████| 89987/89987 [06:52<00:00, 218.14it/s]


In [51]:
##############################################################
################# PLOT ABSOLUTE LOCATIONS X+Y ################
##############################################################
colors_5= ['red','blue', 'cyan', 'green', 'yellow']
names = ['female','male','pup1','pup2','wood block']
for p in range(5):
    temp = animal_arrays[p].sum(1)
    print (temp.shape)
    idx = np.where(temp==0)[0]
    temp[idx]=np.nan
    
    plt.scatter(np.arange(temp.shape[0])/25,temp, 
                c=colors_5[p])
    
    plt.scatter(0,temp[0], 
                c=colors_5[p],
               label=names[p])
    
plt.legend(fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)
plt.show()


(89988,)
(89988,)
(89988,)
(89988,)
(89988,)


In [68]:
##############################################################
################# PLOT VELOCITIES ############################
##############################################################
colors_5= ['red','blue', 'cyan', 'green', 'yellow']
names = ['female','male','pup1','pup2','wood block']
for p in range(4):
    #ax=plt.subplot(4,1,p+1)
    temp = animal_arrays[p]
    temp = temp[1:]-temp[:-1]

    
    temp = np.sqrt(temp[:,0]**2+temp[:,1]**2)
    print (temp.shape)
    #idx = np.where(temp==0)[0]
    #temp[idx]=np.nan
    idx = np.where(temp>300)[0]
    temp[idx]=np.nan
    
    plt.plot(np.arange(temp.shape[0])/25,temp, 
                c=colors_5[p])
    
    plt.scatter(0,temp[0], 
                c=colors_5[p],
               label=names[p])
    
    plt.legend(fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)
plt.show()

(89987,)
(89987,)
(89987,)
(89987,)


In [54]:

video_out.release()
original_vid.release()

In [10]:
# COMPUTE SOCIAL INTERACTIONS BETWEEN ANIMALS

distance_threshold = 25 # # of pixels away
time_window = 1*25 # no of seconds to consider
smoothing_window = 12
min_distance = 25 # number of frames window

print (traces_23hrs[0].shape)
# plt.plot(traces_2D[0][0])
# plt.plot(traces_2D[0][1])

animals=np.arange(4)
interactions = np.zeros((4,4),'int32')
durations = []
for animal1 in animals:
    durations.append([])
    for animal2 in animals:
        durations[animal1].append([])
        if animal1==animal2:
            continue
        traces1=traces_23hrs[animal1].copy()
        traces1[:,0]=np.convolve(traces_23hrs[animal1,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1[:,1]=np.convolve(traces_23hrs[animal1,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1 = traces1
        #print (traces1)

        traces2=traces_23hrs[animal2].copy()
        traces2[:,0]=np.convolve(traces_23hrs[animal2,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces2[:,1]=np.convolve(traces_23hrs[animal2,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces2= traces2

        #print ("traces1: ", traces1.shape)
        # loop over chunks of time
        close = False
        for k in range(min_distance,traces1.shape[0]-min_distance,min_distance):
            dist = np.sqrt((np.mean(traces1[k:k+min_distance,0]) - traces2[-min_distance+k:k+min_distance,0])**2+
                           (np.mean(traces1[k:k+min_distance,1]) - traces2[-min_distance+k:k+min_distance,1])**2)
            #print (dist.shape)
            min_dist = np.min(dist)
            if min_dist<=distance_threshold:
                if close==False:
                    interactions[animal1,animal2]+=1
                    close=True
                    start_time = k
            else:
                if close==True:
                    duration = k-start_time
                    durations[animal1][animal2].append(duration)
                    close=False
        #
plt.imshow(interactions)
plt.show()   
    
    

(2069710, 2)


In [86]:
######################################################
################## No. of interactions ###############
######################################################
clrs=['red','blue','cyan','green']
for k in range(4):
    offset=0
    for p in range(4):
        if k==p:
            continue
        plt.bar(k*1+offset,interactions[k,p],0.2,color=clrs[p])
        offset+=0.2

        
xticks_old = np.arange(0.2,4.2,1)
xticks_new = names
plt.xticks(xticks_old, xticks_new,fontsize=20)
plt.yticks(fontsize=20)

plt.title("Number of inter-animal interactions (23 hour period)",fontsize=20)
plt.show()


In [112]:
######################################################
################## No. of interactions ###############
######################################################

bins = np.arange(0,1000,25)/25
#print (durations[0][2])
for p in range(4):
    for k in range(4):
        if p==k:
            continue
        ax=plt.subplot(1,4,p+1)
        y = np.histogram(np.array(durations[p][k])/25,bins=bins)
        plt.plot(y[1][:-1],y[0],c=clrs[k])
        plt.xlim(0,bins[-1])
        plt.ylim(bottom=0)
        plt.xticks(fontsize=20)
        if p==0:
            plt.xlabel("Duration (sec)",fontsize=20)
    ax.set_yscale('symlog')
    if p>0:
        ax.set_yticks([])
    plt.yticks(fontsize=20)
    plt.title(names[p],fontsize=20)
plt.suptitle("Duration of each inter-animal interaction",fontsize=20)
plt.show()



In [26]:
# BEHAVIUR ANALYSIS - POST TRACE ID SWAP FIXING

data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_corrected.npz',
               allow_pickle=True)
tracesx = data['tracesx']
tracesy = data['tracesy']
print (tracesx)


# plot time series
clrs = ['red','blue','cyan','green']
# from scipy.ndimage import gaussian_filter1d
# from scipy import interpolate

window = 25 * 300 # seconds
#ax=plt.subplot(1,1,1)

ctr=0

# initialize arrays to hold data
# sums_array = []
# sums_individuals = []
# for k in range(4):
#     sums_individuals.append([])
# print ("sums individualsl" , sums_individuals)
# print ("Loading: ", fname, tracesx.shape)

traces = np.zeros((tracesx.shape[0],tracesx.shape[1],2),'float32')
traces[:,:,0]=tracesx
traces[:,:,1]=tracesy
print ("TRACES: ", traces.shape)

idx = np.where(traces==0)
traces[idx]=np.nan

# Load trace info
sums=np.zeros(traces.shape[0]+10,'float32')
for p in range(4):
    
    # find nans and interpolate over them
    # locs = np.nanmean(np.nanmean(np.abs(traces[p*14: (p+1)*14,:,:]),2),0)
    locs = traces[:,p,0]
    x = np.where(np.isnan(locs)==False)[0]
    
    # fill in missing data with linear interpolation
    xnew = np.arange(x[0],x[-1]+1,1)
    fx = interpolate.interp1d(x, traces[x,p,0])
    fy = interpolate.interp1d(x, traces[x,p,1])
    
    ynew1 = fx(xnew)   # use interpolation function returned by `interp1d`
    ynew2 = fy(xnew)   # use interpolation function returned by `interp1d`
    
    dists = np.sqrt((ynew1[1:]-ynew1[:-1])**2+(ynew2[1:]-ynew2[:-1])**2)
    # plt.plot(xnew/25.+ctr*3600, ynew)
    #plt.plot(xnew, ynew)
    # compute differences as movement
    #ynew1 = np.abs(ynew[1:]-ynew[:-1])
    ynew = dists
    
    # instead, just grab all the data and reprocess it below
    #sums[xnew[:ynew.shape[0]]]+=ynew*10
    #print ("sums: ", sums.shape, xnew.shape, ynew.shape)
    sums[xnew[:-1]]+=ynew

    sums_individuals[p] = ynew
    print ("len:", len(sums_individuals[p]))

print ("SUMS: ", sums.shape)
# ynew = np.hstack(sums_array)
ynew = sums

ynew = np.convolve(ynew, np.ones((window,))/window, mode='valid')
print ("ynew: ", ynew.shape)
t = np.arange(ynew.shape[0])/25.+1*3600
# plt.plot(t, ynew, linewidth=3, c='black', label="Sum")

window = 25*1
final_traces = []
for p in range(4):
    print ("plotting animal:", p)
    ynew = sums_individuals[p]
    print ("Ynew: ", ynew)
    ynew = np.convolve(ynew, np.ones((window,))/window, mode='valid')
    t = np.arange(ynew.shape[0])/25.#+1*3600
    final_traces.append(ynew)

print ("Final traces: ", len(final_traces))
    

# label information
plt.ion()
fig, ax = plt.subplots()
r = plt.gcf().canvas.get_renderer()
#x = im.make_image(r, magnification=2.0)


plt.ylabel("Activity", fontsize=30)
plt.yticks([0,10,20,30,40],fontsize=30)
#xticks_old = np.arange(0*3600,3600*24, 3600)
#xticks_new = np.arange(24)
#plt.xticks(xticks_old,xticks_new,fontsize=30)
#plt.xlabel("Time of Day (24hrs clock)",fontsize=30)
# ax.axvspan(0, 7*3600, alpha=0.1, color='black')
# ax.axvspan(19*3600,24*3600, alpha=0.1, color='black')
#plt.xlim(0,60)
#plt.ylim(bottom=0)
plt.xticks(fontsize=30)
plt.xlabel("Time (sec)",fontsize=30)
plt.ylim(0,50)
plt.suptitle("Movement analysis for Cohort #1 - March 16\n"+
             str(window/25)+"sec smoothing window",fontsize=25)

k=0
lines = []
for p in range(4):
    temp, = ax.plot(t[k:k+25*60], final_traces[p][k:k+25*60], linewidth=3, c=clrs[p], label=names[p])
    lines.append(temp)
plt.legend(fontsize=20,loc=1)
    
for k in range(1,3600*25,25):
    for p in range(4):
        lines[p].set_ydata(final_traces[p][k:k+25*60])
        lines[p].set_xdata(np.arange(k,k+25*60,1)/25)
        
    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.xlim(k/25,(k+25*60)/25)
    #.setdata(ynew[k:k+25*60])
    print (k)

plt.show()
    
    



[[       nan 1142.6586         nan  421.74698]
 [       nan 1135.3518         nan  419.19302]
 [       nan 1139.0096         nan  416.82108]
 ...
 [ 300.28156        nan  263.7146         nan]
 [ 301.60367        nan  244.42133  325.44324]
 [ 307.49664        nan  243.933    323.9843 ]]
TRACES:  (89988, 4, 2)


NameError: name 'interpolate' is not defined

In [96]:
for p in range(4):
    temp = np.sum(final_traces[p])
    temp = temp/25 / 100  #convert to meters
    plt.bar(p, temp,0.9, color=clrs[p], label=names[p])
    plt.ylabel("# of Metres traveled in 1 hour",fontsize=30)
plt.legend(fontsize=30,loc=1)
plt.xticks([])
plt.yticks(fontsize=30)
plt.show()

# of lableed frames:  (39225,)
 # of labeled segments:  (1264,)
(229,)
(351,)
(4346,)
(374,)
(9,)
(9,)
(10,)
(4,)
(2,)
(7,)
(5,)
(2,)
(2,)
(4,)
(2,)
(4,)
(7,)
(2,)
(8,)
(1,)
(3,)
(2,)
(20,)
(2,)
(6,)
(2,)
(3,)
(2,)
(3,)
(5,)
(2,)
(3,)
(1,)
(30,)
(4,)
(2,)
(25,)
(5,)
(2,)
(11,)
(9,)
(24,)
(1,)
(4,)
(223,)
(19,)
(1,)
(2,)
(43,)
(1,)
(2,)
(469,)
(2,)
(1,)
(1,)
(2,)
(2,)
(126,)
(3,)
(1,)
(4,)
(1,)
(9,)
(1,)
(3,)
(4,)
(6,)
(2,)
(1,)
(53,)
(5,)
(1,)
(2,)
(1,)
(3,)
(1,)
(1,)
(6,)
(151,)
(50,)
(33,)
(2,)
(5,)
(21,)
(2087,)
(1,)
(122,)
(1,)
(2,)
(4,)
(4,)
(1,)
(1,)
(2,)
(3,)
(1,)
(1,)
(2,)
(6,)
(1,)
(52,)
(3,)
(2,)
(1,)
(1,)
(2,)
(4,)
(26,)
(1,)
(3,)
(8,)
(3,)
(2,)
(4,)
(5,)
(73,)
(4,)
(12,)
(4,)
(15,)
(1,)
(1,)
(1,)
(2,)
(3,)
(4,)
(1,)
(2,)
(16,)
(1,)
(1,)
(132,)
(2,)
(1,)
(5,)
(11,)
(1,)
(9,)
(3,)
(4,)
(1,)
(1,)
(3,)
(4,)
(1,)
(1,)
(4,)
(3,)
(2,)
(14,)
(1,)
(2,)
(1,)
(1,)
(16,)
(9,)
(3,)
(4,)
(12,)
(2,)
(3,)
(9,)
(35,)
(101,)
(2,)
(1,)
(4,)
(1,)
(59,)
(1,)
(66,)
(59,)
(92,)
(2,)
(11,)
(2,)
(6

(9999,)
female 10736 (9522,)
male 11964 (9162,)
pup1 6588 (5776,)
pup2 4121 (3737,)
block 5816 (5740,)
